In [1]:
import sqlite3
import pandas as pd


In [2]:
DB_PATH = "../../../data/benchmark_datasets/disgenet/disgenet_2020.db"


In [3]:
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()


In [12]:

table_name = "diseaseAttributes"

c.execute("SELECT * FROM {}".format(table_name))

rows = c.fetchall()
disease_df = pd.DataFrame(rows, columns=["diseaseNID", "diseaseId", "diseaseName", "type"])


In [13]:
table_name = "geneAttributes"
c.execute("SELECT * FROM {}".format(table_name))

rows = c.fetchall()
gene_df = pd.DataFrame(rows, columns=["geneNID", "geneId", "geneName", "geneDescription", "pLI", "DSI", "DPI"])


In [14]:
table_name = "variantAttributes"
c.execute("SELECT * FROM {}".format(table_name))

rows = c.fetchall()
variant_df = pd.DataFrame(rows, columns=["variantNID", "variantId", "s", "chromosome", "coord", "most_severe_consequence", "DSI", "DPI"])


In [15]:
table_name = "geneDiseaseNetwork"
c.execute("SELECT * FROM {}".format(table_name))

rows = c.fetchall()
disease_gene_df = pd.DataFrame(rows, columns=["NID", "diseaseNID", "geneNID", "source", "association", "associationType", "sentence", "pmid", "score", "EL", "EI", "year"])

# Selecting association with maximum score
disease_gene_df_selected  = disease_gene_df[disease_gene_df.score == 1]

disease_gene_df_selected_1 = pd.merge(disease_gene_df_selected, disease_df, on="diseaseNID")
disease_gene_df_selected_2 = pd.merge(disease_gene_df_selected_1, gene_df, on="geneNID")

disease_gene_df_selected_final = disease_gene_df_selected_2[["diseaseName", "geneName"]].drop_duplicates()
disease_gene_df_selected_final.to_csv("../../../data/benchmark_datasets/disgenet/disease_gene_association_with_score_1_from_disgenet.csv", index=False, header=True)


In [8]:
table_name = "variantDiseaseNetwork"
c.execute("SELECT * FROM {}".format(table_name))

rows = c.fetchall()
disease_variant_df = pd.DataFrame(rows, columns=["NID", "diseaseNID", "variantNID", "source", "association", "associationType", "sentence", "pmid", "score", "EI", "year"])

# Selecting association with score > 0.9
disease_variant_df_selected  = disease_variant_df[disease_variant_df.score > 0.9]

disease_variant_df_selected_1 = pd.merge(disease_variant_df_selected, disease_df, on="diseaseNID")
disease_variant_df_selected_2 = pd.merge(disease_variant_df_selected_1, variant_df, on="variantNID")

disease_variant_df_selected_final = disease_variant_df_selected_2[["diseaseName", "variantId"]].drop_duplicates()

disease_variant_df_selected_final.to_csv("../../../data/benchmark_datasets/disgenet/disease_variant_association_with_score_greater_than_point_9_from_disgenet.csv", index=False, header=True)

